In [55]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate,AIMessagePromptTemplate
from langchain_core.messages import (
    SystemMessage,
    AIMessage,
    HumanMessage
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

### 表决PromptTemplate

In [323]:
class EvaluateAgentResponseRromptTemplate():
    def generate_vote_template(self):
        #task: str,contexts: str,agent_response: str
        return """
        Based on the context and your characterization, please vote for the task response. You must only answer True or False, don't say anything else:
        The following are task, contexts, and it's response given:
        Contexts: {contexts}
        Task: {task}
        Response: {agent_response}
        True indicates approval of the response, Flase indicates disapproval:
        Your Choice(Only True or False):
        """

### Base Agent for 评分、表决、执行

In [324]:
# 评分Agent
class BaseAgent:  
    def __init__(self,
                 name: str,
                 llmName: str,
                 roleDescription: str,
                 task: str,
                 contexts: str = ""):  # 通过——进行评分
        self.name = name
        self.llmModel = ChatOllama(model=llmName)
        self.roleDescription = roleDescription
        self.task = task
        self.contexts = contexts
        self.history_messages = []
        self.evaluate_scoreResponse = ""# 专门保存打分任务得分回复
        self.responses = []
        # 创建时，就确定其身份
        sysTemplate = self.roleDescription
        sysMsgPrompt = SystemMessagePromptTemplate.from_template(sysTemplate)
        self.add_to_history(sysMsgPrompt)
        
    def add_to_history(self, message):
        self.history_messages.append(message)
        
    def generate_prompt(self, messages):
        chat_prompt = ChatPromptTemplate.from_messages(messages)
#         print(chat_prompt)
#         tmp = chat_prompt.format_prompt(task=self.task, sentence = self.sentence).to_messages()
#         print(tmp)
        return chat_prompt

    def get_evaluate_score(self):
        self.history_messages = self.history_messages[:1]
        humTemplate = "Next, you're going to complete task for the following context in a way that's characteristic of your identity. \n Task: {task} \n Context: {contexts} \n You must only ansower evoluation score,don't say anything else. \n Evoluation Score:"
        humMsgPrompt = HumanMessagePromptTemplate.from_template(humTemplate)
        self.add_to_history(humMsgPrompt)
        
        prompt = self.generate_prompt(self.history_messages)
#         print(prompt)
        chain = prompt | self.llmModel | StrOutputParser()
        evaluate_score = chain.invoke({
            "task": self.task,
            "contexts": self.contexts
        }).strip()
        self.evaluate_scoreResponse = evaluate_score
        return self.evaluate_scoreResponse #response,
    
    def execute_task(self):
        self.history_messages = self.history_messages[:1]
        humTemplate = "Next, you're going to complete task for the following context in a way that's characteristic of your identity. \n Task: {task} \n Context: {contexts} \n Response:"
        humMsgPrompt = HumanMessagePromptTemplate.from_template(humTemplate)
        self.add_to_history(humMsgPrompt)
        
        prompt = self.generate_prompt(self.history_messages)
#         print(prompt)
        chain = prompt | self.llmModel | StrOutputParser()
        response = chain.invoke({
            "task": self.task,
            "contexts": self.contexts
        }).strip()
        self.responses.append(response)
        return response #response,
    
    def execute_vote_task(self, task:str, agent_response: str):
        self.history_messages = self.history_messages[:1]
        humTemplate = EvaluateAgentResponseRromptTemplate()
        humTemplate = humTemplate.generate_vote_template()
        humMsgPrompt = HumanMessagePromptTemplate.from_template(humTemplate)
        self.add_to_history(humMsgPrompt)
        prompt = self.generate_prompt(self.history_messages)
        vote_chain = prompt | self.llmModel | StrOutputParser()
        choice_response = vote_chain.invoke({
            "task": task,
            "contexts": self.contexts,
            "agent_response": agent_response
        }).strip()
        return choice_response

In [325]:
INFJAgent = BaseAgent(name = "INFJ",
                      llmName="qwen:7b",
                      roleDescription="You are the quiet visionary, usually inspiring and tireless idealist.",
                      task="fluency evaluation(evaluate on a scale of 1-5)", #
                      contexts="Knowledge can change your fate and English can accomplish your future."
                      )
INFJres = INFJAgent.get_evaluate_score()


INFPAgent = BaseAgent(name = "INFP",
                      llmName="qwen:7b",
                      roleDescription="You are poetic, kind and selfless, always eager to help a just cause.",
                      task="fluency evaluation(evaluate on a scale of 1-5)", #
                      contexts="Knowledge can change your fate and English can accomplish your future."
                      )
INFPres = INFPAgent.get_evaluate_score()

In [326]:
print(INFJres)
print("-----------")
print(INFPres)

4
-----------
4


In [322]:
INFJAgent.name

'INFJ'

In [327]:
INFJAgent.evaluate_scoreResponse

'4'

In [328]:
INFPAgent.evaluate_scoreResponse

'4'

### 任务/权重分配Assign Agent

In [82]:
import json
# from typing import Optional,List

In [139]:
# 任务分配Assign Agent
class AssignAgent(BaseAgent):
    def __init__(self,
                 roleDescription: str,
                 task: str,
                 subWorkNumber: int = 1,
                 llmName: str = "qwen:7b",
                ):
        #                  subWorks: Optional(List[str]) = None,
        super().__init__(llmName=llmName,
                        roleDescription=roleDescription,
                        task=task)
        # 若让Agent生成子任务
#         self.subWorks = subWorks
        self.subWorkNumber = subWorkNumber
        self.descriptionOfGroups = [{
            "Name": "SP-type people ",
            "description": "They know that what they are doing now is to get what they want more smoothly and efficiently.",
            "type": "object",
            "properties": {
                "Skills": {
                    "title": "Skills",
                    "description": "Skills these people good at.",
                    "type": "string",
                    "content":"""
                        1.Highly adaptable and motor-neurotic;
                        2.Understands the reality of the situation and never fights it;
                        3.Being observant and aware of what is happening around them;
                        4.A preference for practicality, storing useful factual information with less regard for theoretical knowledge;
                        5.Easy-going and patient, with a tendency to act in a cost-effective manner;
                        6.An aptitude for working with machinery and tools;
                        7.Sensitivity to color, line and structure, artistic temperament;
                        8.Desire for personal experience in all things and know how to enjoy life.
                    """
                },
            },
        },{
            "Name": "SJ-type people ",
            "description": "To be true to the point that everything should be done in a way that is appropriate and acceptable to the people.",
            "type": "object",
            "properties": {
                "Skills": {
                    "title": "Skills",
                    "description": "Skills these people good at.",
                    "type": "string",
                    "content":"""
                        1.Conservative and reliable, following rules and regulations
                        2.Perceptive and decisive
                        3.Practical, never impulsive
                        4.Patient and trustworthy
                        5.Hard-working and conscientious
                        6.Rigorous, detail-oriented and persistent

                    """
                },
            },
        },{
            "Name": "NF-type people ",
            "description": "One thing is very important to them: they want all the people around them to feel good about them and to be happy with them.",
            "type": "object",
            "properties": {
                "Skills": {
                    "title": "Skills",
                    "description": "Skills these people good at.",
                    "type": "string",
                    "content":"""
                        1.Kind and compassionate;
                        2.Pious and passionate;
                        3.Possessing keen intuition;
                        4.Being insightful;
                        5.Extremely creative;
                        6.Highly subjective.
                    """
                },
            },
        },{
            "Name": "NT-type people ",
            "description": "Everything they do and say makes sense.",
            "type": "object",
            "properties": {
                "Skills": {
                    "title": "Skills",
                    "description": "Skills these people good at.",
                    "type": "string",
                    "content":"""
                        1.Good at analyzing and being organized;
                        2.Abstract and theoretical views;
                        3.Rational and complex ideas;
                        4.Strong ability and creativity;
                        5.Highly efficient and rigorous;
                        6.Logical and professional;
                        7.Curious and good at research
                    """
                },
            },
        }
        ]
        self.response_schema = {
            "title": "Groups",
            "description": "Identifying information about a group.",
            "type": "object",
            "properties": {
                "name": {"title": "Name", "description": "The group's name", "type": "string"},
                "weights": {
                    "title": "Weights",
                    "description": "The weight you assigned to this group based on the task",
                    "type": "float",
                },
                "reason": {
                    "title": "Reason",
                    "description": "Reasons for assigning this weight to this group",
                    "type": "str",
                },
#                 "tasked": {
#                     "title": "tasks",
#                     "description": "the subtasks you break task down rationally into.",
#                     "type": "string",
#                 },
            },
            "required": ["name", "weights"],
        }
        self.json_responseScheme = json.dumps(self.response_schema, indent=2)
        self.step_schema = {
            "title": "Evaluation Step",
            "description": "Identifying information about this evaluation step.",
            "type": "object",
            "properties": {
                "id": {"title": "ID", "description": "The this step ID", "type": "string"},
                "reason": {
                    "title": "Reason",
                    "description": "Reasons why this step is set up this way.",
                    "type": "str",
                },
                "content": {
                    "title": "Content",
                    "description": "A brief description of the content of this evaluation step.",
                    "type": "string",
                },
                "weight": {
                    "title": "Weight",
                    "description": "Quantitative description of the importance of this assessment step in the overall task",
                    "type": "float",
                },
            },
            "required": ["id", "reason", "content", "weight"],
        }
        self.json_stepScheme = json.dumps(self.step_schema, indent=2)

    def generate_prompt(self, messages):
        chat_prompt = ChatPromptTemplate.from_messages(messages)
#         print(chat_prompt)
#         tmp = chat_prompt.format_prompt(
#             task= self.task,
#             subWorkNumber = self.subWorkNumber,
#             descriptionOfGroups = self.descriptionOfGroups,
#             json_responseScheme = self.json_responseScheme
#         ).to_messages()
#         print(tmp)
        return chat_prompt
    
    def execute_task(self):
        sysTemplate = self.roleDescription
        sysMsgPrompt = SystemMessagePromptTemplate.from_template(sysTemplate)
        self.add_to_history(sysMsgPrompt)
        ####################步骤划分#################################
        humTemplate_forStepGenerate="""
        You need to Divide the task of {task}  rationally into {subWorkNumber} evaluation steps to complete in a way that's characteristic of your identity, and assigns a weight to this evaluation step based on its importance in the overall task, with the sum of the weights of all evaluation steps for a task being 1.
        Note that the process must be oriented to get the result of the task without any additional operations.
        Please tell me about these steps just only using the following JSON schema, you must only answer the json,don't say anything else:
        {json_stepScheme}        
        """
        humMsgPrompt_forStepGenerate = HumanMessagePromptTemplate.from_template(humTemplate_forStepGenerate)
        self.add_to_history(humMsgPrompt_forStepGenerate)
        prompt_for_steps = self.generate_prompt(self.history_messages)      
        chain_for_steps = prompt_for_steps | self.llmModel | StrOutputParser()
        steps_response = chain_for_steps.invoke({
            "task": self.task,
            "subWorkNumber": self.subWorkNumber,
            "json_stepScheme": self.json_stepScheme
        }).strip()
        self.history_messages.pop()
        ####################权重分配#################################
        humTemplate = """
        Based on the task of {task}, you need to judge following groups fitness for this task and assign them weights (the sum of the weights is 1) in a way that's characteristic of your identity.
        A description of the four groups is provided: 
        {descriptionOfGroups}
        Please tell me about these groups just only using the following JSON schema, you must only answer the json,don't say anything else:
        {json_responseScheme}
        """
        humMsgPrompt = HumanMessagePromptTemplate.from_template(humTemplate)
        self.add_to_history(humMsgPrompt)     
        prompt = self.generate_prompt(self.history_messages)
        chain = prompt | self.llmModel | StrOutputParser()

        response = chain.invoke({
            "task": self.task,
            "descriptionOfGroups": self.descriptionOfGroups,
            "json_responseScheme": self.json_responseScheme
        }).strip()
        return steps_response, response #response,

In [140]:
assignAgent = AssignAgent(
                llmName="qwen:7b",
                roleDescription="You are a professional psychologist and co-ordinator who specializes in subtask generation and assigning tasks based on character traits",
                task="fluency ratings(evaluate on a scale of 1-5)", #
#                 sentence="Knowledge can change your fate and English can accomplish your future."
                subWorkNumber = 2
)

In [141]:
step,res = assignAgent.execute_task()

In [142]:
print(step)

{
"title": "Evaluation Step 1: Fluency Rating",
"id": "step1",
"reason": "Initial assessment step for overall fluency rating.",
"content": "Rate the speaker's ability to produce speech fluently. Scale from 1-5, with 1 being poor fluency and 5 being excellent fluency.",
"weight": 0.6
},
{
"title": "Evaluation Step 2: Fluency Duration",
"id": "step2",
"reason": "Second assessment step to evaluate the speaker's sustained fluency.",
"content": "Measure how long the speaker maintains fluent speech without pauses or breaks. This can be done by time-stamping specific sections of the speech.",
"weight": 0.4
}
]


In [149]:
import re
steps_match = re.search(r'\{.*\}', step, re.DOTALL)

if steps_match:
    steps_result = steps_match.group(0)
    steps_result = "["+steps_result+"]"
    print(steps_result)
else:
    print("No match found")

[{
"title": "Evaluation Step 1: Fluency Rating",
"id": "step1",
"reason": "Initial assessment step for overall fluency rating.",
"content": "Rate the speaker's ability to produce speech fluently. Scale from 1-5, with 1 being poor fluency and 5 being excellent fluency.",
"weight": 0.6
},
{
"title": "Evaluation Step 2: Fluency Duration",
"id": "step2",
"reason": "Second assessment step to evaluate the speaker's sustained fluency.",
"content": "Measure how long the speaker maintains fluent speech without pauses or breaks. This can be done by time-stamping specific sections of the speech.",
"weight": 0.4
}]


In [150]:
res_match = re.search(r'\{.*\}', res, re.DOTALL)

if res_match:
    res_result = res_match.group(0)
    res_result = "["+res_result+"]"
    print(res_result)
else:
    print("No match found")

[{ "name": "SP-people Group", "weights": 0.3, "reason": "High adaptability and motor-neurotic traits make them skilled in fluency ratings." },
{ "name": "SJ-people Group", "weights": 0.4, "reason": "Conservative and reliable nature makes them trustworthy for evaluating fluency. Decision-making abilities also contribute to their effectiveness in this task." },
{ "name": "NF-people Group", "weights": 0.25, "reason": "Their empathetic and artistic disposition make them skilled at connecting with others and creating engaging content, which can be reflected in fluency ratings." },
{ "name": "NT-people Group", "weights": 0.3, "reason": "Their strong analytical skills and organized nature make them particularly effective at evaluating fluency. Their creativity and complex ideas also contribute to their unique perspective in this task." }]


### 组内协商

In [344]:
import pandas as pd
import numpy as np
class IntraGroupNegotiation():
    def __init__(self,
                 agents: List[BaseAgent],
                 steps: str,  #json列表字符串
#                  groups: str,  #json列表字符串  #迁移到InterNegotiation中
                 ):
        self.agents = agents
        self.steps = json.loads(steps)
        self.groups = json.loads(groups)
        self.epochs = len(self.steps)  # 协商轮数
        self.vote2score_info = {} #最终组内协商后的加权评分信息
        sum_stepWeights = 0
        sum_groupWeights = 0

        # 更新权重，因为可能LLM给出的权重和不一定为1
        for step in self.steps:
            sum_stepWeights += step["weight"]
        for group in self.groups:
            sum_groupWeights += group["weights"]
        
        for step in self.steps:
            step["weight"] = step["weight"]/sum_stepWeights
        for group in self.groups:
            group["weights"] = group["weights"]/sum_groupWeights
     
    
    def run_negotiate(self):
        print("EPOCH:",self.epochs)
        print(self.steps)
        agentsNum = len(self.agents)
        dfs = {}
        task_overall_responses=[]
        # 开始进行组内协商
        for epoch in range(self.epochs):
            print("epoch:",epoch)
            response_of_agents = [] # 用于该step中agent的相应
            task_of_step = self.steps[epoch]["content"]
            vote_df = pd.DataFrame(index=[agent.name for agent in self.agents], 
                                    columns=[agent.name for agent in self.agents])     # 记录每个Step中投票情况       
            for agent in self.agents:
                # 生成每个agent关于该step的响应
                agent.task = task_of_step
                res = agent.execute_task()
                response_of_agents.append(res)
                
            for idx, agent in enumerate(self.agents):
                # 生成该轮讨论下，agents间对彼此的投票
                for i in range(agentsNum):
                    if i != idx:
                        vote_res = agent.execute_vote_task(task = task_of_step, agent_response = response_of_agents[i])                      
                        vote_df.loc[self.agents[idx].name, self.agents[i].name] = vote_res
                        
            task_overall_responses.append(response_of_agents)  # 每一轮中每个agent的回答详情
            dfs["step"+str(epoch+1)] = vote_df.applymap(lambda x: 1 if x == 'True' else 0)  # 替换掉True,False,NAN
            
        return dfs, task_overall_responses
    
    def cal_score(self,
                  voteInfo: dict,
                  ) -> dict:
#         vote2score_info = {}
        vote_with_steps = []
        for epoch in range(self.epochs):
            vote_with_steps.append(voteInfo["step"+str(epoch+1)].sum().to_frame().T)
        vote_with_steps = pd.concat(vote_with_steps,ignore_index=True)
        vote_with_steps.index = ["step"+str(x+1) for x in range(self.epochs)]  # steps-agents票数记录,要返回
        
        # 找出每个step中得票数最多的Agent
        maxVotedAgent_eachStep = []
        for _, row in vote_with_steps.iterrows():
            max_votes = max(row)
            top_agents = [agent for agent, vote in row.items() if vote == max_votes]
            if max_votes == 0:
                top_agents = []
            maxVotedAgent_eachStep.append(top_agents)  # [[],['Agent2'], ['Agent1', 'Agent2']]
            
        #计算最终加权评分
        agents_evaluationScores = {}  # 每个agent对任务的宏观打分
        for agent in self.agents:
            agents_evaluationScores[agent.name] = int(agent.evaluate_scoreResponse)

        step_weights = [step["weight"] for step in self.steps]            #每一步的权重
        step_value_from_agents = []  # Agents联合为每一步打的分
        tmp_sum=0
        final_score=0 #组内协商后最终评估得分
        for item in maxVotedAgent_eachStep:
            if not item:
                step_value_from_agents.append(0)
            else:
                for inner_item in item:
                    tmp_sum += agents_evaluationScores[inner_item]
                step_value_from_agents.append(tmp_sum/len(item))
                tmp_sum=0
                
        step_value_from_agents = np.array(step_value_from_agents)
        step_weights = np.array(step_weights)
        final_score = np.dot(step_value_from_agents, step_weights)  # 点积计算加权得分
        self.vote2score_info["vote_detailed"] = vote_with_steps
        #   Agent1  Agent2
        #step1  1    1
        #step2  0    0
        self.vote2score_info["maxVotedAgent_eachStep"] =  maxVotedAgent_eachStep
        # [[],['Agent2'], ['Agent1', 'Agent2']]
        self.vote2score_info["step_value_from_agents"] = step_value_from_agents
        #[0,1]
        self.vote2score_info["step_weights"] = step_weights
        #[0.4,0.6]
        self.vote2score_info["intra_group_final_score"] = final_score 
        # 0.6
        return self.vote2score_info

In [345]:
negotiation = IntraGroupNegotiation(agents=[INFJAgent,INFPAgent],
                               steps=steps_result,
                               groups=res_result)

In [338]:
neg_dfs, overall_respionses = negotiation.run_negotiate()

EPOCH: 2
[{'title': 'Evaluation Step 1: Fluency Rating', 'id': 'step1', 'reason': 'Initial assessment step for overall fluency rating.', 'content': "Rate the speaker's ability to produce speech fluently. Scale from 1-5, with 1 being poor fluency and 5 being excellent fluency.", 'weight': 0.6}, {'title': 'Evaluation Step 2: Fluency Duration', 'id': 'step2', 'reason': "Second assessment step to evaluate the speaker's sustained fluency.", 'content': 'Measure how long the speaker maintains fluent speech without pauses or breaks. This can be done by time-stamping specific sections of the speech.', 'weight': 0.4}]
epoch: 0
epoch: 1


In [348]:
print(neg_dfs["step1"])
print(neg_dfs["step2"])

      INFJ  INFP
INFJ     0     1
INFP     1     0
      INFJ  INFP
INFJ     0     1
INFP     1     0


In [346]:
vote2score_info = negotiation.cal_score(neg_dfs)

In [347]:
vote2score_info

{'vote_detailed':        INFJ  INFP
 step1     1     1
 step2     1     1,
 'maxVotedAgent_eachStep': [['INFJ', 'INFP'], ['INFJ', 'INFP']],
 'step_value_from_agents': array([4., 4.]),
 'step_weights': array([0.6, 0.4]),
 'intra_group_final_score': 4.0}

### 组间共谋

In [368]:
#groupsAssign---[{ "name": "SP-people Group", "weights": 0.3, "reason": "High adaptability and motor-neurotic traits make them skilled in fluency ratings." },
#{ "name": "SJ-people Group", "weights": 0.4, "reason": "Conservative and reliable nature makes them trustworthy for evaluating fluency. Decision-making abilities also contribute to their effectiveness in this task." },
#{ "name": "NF-people Group", "weights": 0.25, "reason": "Their empathetic and artistic disposition make them skilled at connecting with others and creating engaging content, which can be reflected in fluency ratings." },
#{ "name": "NT-people Group", "weights": 0.3, "reason": "Their strong analytical skills and organized nature make them particularly effective at evaluating fluency. Their creativity and complex ideas also contribute to their unique perspective in this task." }]
class InterGroupsNegotiation():
    def __init__(self,
                 negotiations: List[IntraGroupNegotiation],
                 groupsAssign: str,  #json列表字符串
                 ):
        """
        Note: the order of groupsAssign should responding to the order of negotiations
              ,that avoid the weights Assigned mistakely.
        """
        self.negotiations = negotiations
        self.groupsAssign = json.loads(groupsAssign)
        self.negotiationNum = len(self.negotiations)
        
        sum_groupWeights = 0
        
        self.eachNegtiationInfo = {}  # 记录每个group negotiation的协商信息
        self.groupsNegotiation2Score_Info = {} # 记录组间协商情况和最终得分
        
        # 更新权重，因为可能LLM给出的权重和不一定为1
        for group in self.groupsAssign:
            sum_groupWeights += group["weights"]
        
        for group in self.groupsAssign:
            group["weights"] = group["weights"]/sum_groupWeights
            
    def run_negotiations(self):
        
        for _, neg in enumerate(self.negotiations):
            neg_dfs, overall_respionses  = neg.run_negotiate()
            self.eachNegtiationInfo["GroupNegotiation"+str(_+1)] = neg.cal_score(neg_dfs)
        
        return self.eachNegtiationInfo  # 每一个intra group negatiation的结果信息
    
    def cal_groups_score(self,
                         eachNegtiationInfo):
        groupScores = [eachNegtiationInfo["GroupNegotiation"+str(_+1)]['intra_group_final_score'] for _ in range(self.negotiationNum)]
        print(groupScores)
        groupWeights = [group["weights"] for group in self.groupsAssign]
        print(groupWeights)
        np_groupScores = np.array(groupScores)
        np_groupWeights = np.array(groupWeights)
        
        self.groupsNegotiation2Score_Info["GroupNegotiations"] = eachNegtiationInfo
        self.groupsNegotiation2Score_Info["GroupWeights"] = groupWeights
        self.groupsNegotiation2Score_Info["InterGroupsScore"] = np.dot(np_groupScores, np_groupWeights)
        return self.groupsNegotiation2Score_Info

In [369]:
negs = [negotiation]
groupsAssign = '[{ "name": "SP-people Group", "weights": 0.3, "reason": "High adaptability and motor-neurotic traits make them skilled in fluency ratings." }]'

In [370]:
inter_neg = InterGroupsNegotiation(negs,groupsAssign)

In [367]:
eachNegtiationInfo = inter_neg.run_negotiations()

EPOCH: 2
[{'title': 'Evaluation Step 1: Fluency Rating', 'id': 'step1', 'reason': 'Initial assessment step for overall fluency rating.', 'content': "Rate the speaker's ability to produce speech fluently. Scale from 1-5, with 1 being poor fluency and 5 being excellent fluency.", 'weight': 0.6}, {'title': 'Evaluation Step 2: Fluency Duration', 'id': 'step2', 'reason': "Second assessment step to evaluate the speaker's sustained fluency.", 'content': 'Measure how long the speaker maintains fluent speech without pauses or breaks. This can be done by time-stamping specific sections of the speech.', 'weight': 0.4}]
epoch: 0
epoch: 1


In [371]:
eachNegtiationInfo

{'GroupNegotiation1': {'vote_detailed':        INFJ  INFP
  step1     1     1
  step2     1     1,
  'maxVotedAgent_eachStep': [['INFJ', 'INFP'], ['INFJ', 'INFP']],
  'step_value_from_agents': array([4., 4.]),
  'step_weights': array([0.6, 0.4]),
  'intra_group_final_score': 4.0}}

In [372]:
groupsNegotiation2Score_Info = inter_neg.cal_groups_score(eachNegtiationInfo)

[4.0]
[1.0]


In [374]:
groupsNegotiation2Score_Info

{'GroupNegotiations': {'GroupNegotiation1': {'vote_detailed':        INFJ  INFP
   step1     1     1
   step2     1     1,
   'maxVotedAgent_eachStep': [['INFJ', 'INFP'], ['INFJ', 'INFP']],
   'step_value_from_agents': array([4., 4.]),
   'step_weights': array([0.6, 0.4]),
   'intra_group_final_score': 4.0}},
 'GroupWeights': [1.0],
 'InterGroupsScore': 4.0}

In [375]:
groupsNegotiation2Score_Info['InterGroupsScore']

4.0

In [277]:
# 计算最中得票数
neg_dfs

{'step1':         Agent1  Agent2
 Agent1       0       2
 Agent2       0       0,
 'step2':        Agent1 Agent2
 Agent1    NaN   True
 Agent2  False    NaN}

In [306]:
neg_dfs

{'step1':         Agent1  Agent2
 Agent1       0       0
 Agent2       0       0,
 'step2':         Agent1  Agent2
 Agent1       0       1
 Agent2       0       0}

In [307]:
cal_score(neg_dfs)

,Agent1,Agent2
step1,0,0
step2,0,1


In [311]:
# 创建样例数据
data = {
    'Agent1': [0, 1],
    'Agent2': [1, 1]
}
df = pd.DataFrame(data, index=['step1', 'step2'])

# 找出每个step中得票数最多的Agent
result = []
for _, row in df.iterrows():
    max_votes = max(row)
    top_agents = [agent for agent, vote in row.items() if vote == max_votes]
    result.append(top_agents)



print(result)

[['Agent2'], ['Agent1', 'Agent2']]


In [315]:
int('Agent2'[-1])-1

1

In [272]:
neg_dfs["step1"].loc["Agent1","Agent2"]

'True'

In [275]:
neg_dfs["step1"] = neg_dfs["step1"].applymap(lambda x: 1 if x == 'True' else 0)
neg_dfs["step1"].loc["Agent1","Agent2"]=2

In [278]:
neg_dfs["step1"]

,Agent1,Agent2
Agent1,0,2
Agent2,0,0


In [280]:
type(neg_dfs["step1"].sum())

pandas.core.series.Series

In [284]:
# 假设你有3个DataFrame
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [2, 3, 4], 'B': [5, 6, 7]})
df3 = pd.DataFrame({'A': [3, 4, 5], 'B': [6, 7, 8]})

# 将结果汇总到一个新的DataFrame
result_df = pd.concat([df1.sum().to_frame().T,
                      df2.sum().to_frame().T,
                      df3.sum().to_frame().T], 
                     ignore_index=True)
result_df.index = ['df1', 'df2', 'df3']

print(result_df)

      A   B
df1   6  15
df2   9  18
df3  12  21


In [257]:
import pandas as pd

# 假设有以下DataFrame
data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
df = pd.DataFrame(data)

# 计算每一列的总和
column_sums = df.sum()

print(column_sums)

A     6
B    15
C    24
dtype: int64


In [212]:

vote_df = pd.DataFrame(index=[f'Agent{i+1}' for i in range(4)], 
                                           columns=[f'Agent{i+1}' for i in range(4)])
vote_df.head()

,Agent1,Agent2,Agent3,Agent4
Agent1,NaN,NaN,NaN,NaN
Agent2,NaN,NaN,NaN,NaN
Agent3,NaN,NaN,NaN,NaN
Agent4,NaN,NaN,NaN,NaN


In [216]:
vote_df.loc['Agent2', 'Agent2']="hello"

In [217]:
vote_df.head()

,Agent1,Agent2,Agent3,Agent4,"(Agent2, Agent2)"
Agent1,NaN,NaN,NaN,NaN,hello
Agent2,NaN,hello,NaN,NaN,hello
Agent3,NaN,NaN,NaN,NaN,hello
Agent4,NaN,NaN,NaN,NaN,hello


In [192]:
"ahent"+str(1)

'ahent1'

In [182]:
groupNeg = GroupNegotiation(steps_result, res_result)

1.0
1.25
[{'title': 'Evaluation Step 1: Fluency Rating', 'id': 'step1', 'reason': 'Initial assessment step for overall fluency rating.', 'content': "Rate the speaker's ability to produce speech fluently. Scale from 1-5, with 1 being poor fluency and 5 being excellent fluency.", 'weight': 0.6}, {'title': 'Evaluation Step 2: Fluency Duration', 'id': 'step2', 'reason': "Second assessment step to evaluate the speaker's sustained fluency.", 'content': 'Measure how long the speaker maintains fluent speech without pauses or breaks. This can be done by time-stamping specific sections of the speech.', 'weight': 0.4}]
[{'name': 'SP-people Group', 'weights': 0.24, 'reason': 'High adaptability and motor-neurotic traits make them skilled in fluency ratings.'}, {'name': 'SJ-people Group', 'weights': 0.32, 'reason': 'Conservative and reliable nature makes them trustworthy for evaluating fluency. Decision-making abilities also contribute to their effectiveness in this task.'}, {'name': 'NF-people Grou